In [ ]:
import os
import torch
from torch import nn
import sys
sys.path.append('..')
from tools import tool_pytorch_017 as ty

#@save
ty.DATA_HUB['aclImdb'] = (
    'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz',
    '01ada507287d82875905620988597833ad4e0903')

data_dir = ty.download_extract('aclImdb', 'aclImdb')

#@save
def read_imdb(data_dir, is_train):
    """读取IMDb评论数据集文本序列和标签"""
    data, labels = [], []
    for label in ('pos', 'neg'):
        folder_name = os.path.join(data_dir, 'train' if is_train else 'test',
                                   label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '')
                data.append(review)
                labels.append(1 if label == 'pos' else 0)
    return data, labels

train_data = read_imdb(data_dir, is_train=True)
print('训练集数目：', len(train_data[0]))
for x, y in zip(train_data[0][:3], train_data[1][:3]):
    print('标签：', y, 'review:', x[0:60])
    
# 将每个单词作为一个词元，过滤掉出现不到5次的单词，我们从训练数据集中创建一个词表。
train_tokens = ty.tokenize(train_data[0], token='word')
vocab = ty.Vocab(train_tokens, min_freq=5, reserved_tokens=['<pad>'])

ty.set_figsize()
ty.plt.xlabel('# tokens per review')
ty.plt.ylabel('count')
ty.plt.hist([len(line) for line in train_tokens], bins=range(0, 1000, 50));

# 为了每次处理一小批量这样的评论，我们通过截断和填充将每个评论的长度设置为500。
num_steps = 500  # 序列长度
train_features = torch.tensor([ty.truncate_pad(
    vocab[line], num_steps, vocab['<pad>']) for line in train_tokens])
print(train_features.shape)

train_iter = ty.load_array((train_features,
    torch.tensor(train_data[1])), 64)

for X, y in train_iter:
    print('X:', X.shape, ', y:', y.shape)
    break
print('小批量数目：', len(train_iter))

#@save
def load_data_imdb(batch_size, num_steps=500):
    """返回数据迭代器和IMDb评论数据集的词表"""
    data_dir = ty.download_extract('aclImdb', 'aclImdb')
    train_data = read_imdb(data_dir, True)
    test_data = read_imdb(data_dir, False)
    train_tokens = ty.tokenize(train_data[0], token='word')
    test_tokens = ty.tokenize(test_data[0], token='word')
    vocab = ty.Vocab(train_tokens, min_freq=5)
    train_features = torch.tensor([ty.truncate_pad(
        vocab[line], num_steps, vocab['<pad>']) for line in train_tokens])
    test_features = torch.tensor([ty.truncate_pad(
        vocab[line], num_steps, vocab['<pad>']) for line in test_tokens])
    train_iter = ty.load_array((train_features, torch.tensor(train_data[1])),
                                batch_size)
    test_iter = ty.load_array((test_features, torch.tensor(test_data[1])),
                               batch_size,
                               is_train=False)
    return train_iter, test_iter, vocab
